# Import

In [5]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "rag-rohto"

# Document Loader

In [32]:
from langchain_community.document_loaders import PyPDFLoader
import pdfplumber

def load_pdf_document(file_path):
    documents = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            documents.append({"content": page.extract_text(), "metadata": {"source": file_path}})
    return documents

file_paths = [
    "../dataset/Financial_Statements_2023.pdf",
    "../dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf",
    "../dataset/Shibata_et_al_Research_Article.pdf",
    "../dataset/V_Rohto_Premium_Product_Information.pdf",
    "../dataset/Well-Being_Report_2024.pdf",
    ]

# PDFファイルを読み込む
docs = [load_pdf_document(file_path) for file_path in file_paths]
# リストをフラット化
docs_list = [item for sublist in docs for item in sublist]

# Document transformer

In [37]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# `docs_list` を Document 型に変換
docs_list_converted = [
    Document(page_content=doc["content"], metadata=doc["metadata"])
    for doc in docs_list
]

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size= 1000,
    chunk_overlap=100,
)

doc_splits = text_splitter.split_documents(docs_list_converted)
print(len(doc_splits))

Created a chunk of size 1097, which is longer than the specified 1000


514


# Embedding model

In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vector store

In [45]:
from langchain_chroma import Chroma

db = Chroma.from_documents(doc_splits, embeddings)

# Retriever

In [46]:
retriever = db.as_retriever()

In [47]:
query = "肌ラボ 極潤ヒアルロン液の使用上の注意点を教えてください。"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'source': '../dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf'}
2024/12/06 8:55 肌ラボ 極潤ヒアルロン液 | ロート製薬: 商品情報サイト
アイケア スキンケア 外⽪薬 内服薬 検査薬 ⿐・⼝腔ケア ⾷品・サプリメント お役⽴ち情報
肌ラボ
化粧品
肌ラボ 極潤ヒアルロン液
発酵技術で⽣まれたうるおい成分 乳酸球菌／ヒア
ルロン酸発酵液 配合。やさしいとろみでうるおっ
て吸いつくようなもちもち肌に
内容量 価格
ボトル 170mL オープン価格
つめかえ⽤ 170mL オープン価格
ポンプ 400mL オープン価格
※希望⼩売価格・税込
肌ラボ 極潤ヒアルロン液の成分表⽰はこちらをご覧ください。
成分表⽰ 販売店検索はこちら
JANコード
肌ラボ 極潤ヒアルロン液について
肌が本当に求めるうるおいを追求した、「極潤」。うるおい成分であるヒアルロン酸にこだわり抜き、必要のない
ものはできる限り削ぎ落として、配合成分と容器をシンプルにしました。
「極潤ヒアルロン液」は、肌が求めるうるおいを詰めこんだ化粧⽔。4種類のヒアルロン酸（うるおい成分）＊
を配合しました。やさしいとろみで、たっぷりうるおい、使うたび、健やかな素肌へ。顔・体・髪の⽑にも。⽼
若男⼥、誰でも使えます。
弱酸性。無⾹料・無着⾊・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。敏感肌の⽅の協
⼒によるパッチテスト済み。（すべての⽅に刺激が起こらないというわけではありません。）
本製品の容器には、環境に配慮したバイオマス原料を⼀部使⽤しています。


# LCELを使ったRAGのChainの実装

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [51]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = "肌ラボ 極潤ヒアルロン液の使用上の注意点を教えてください。"

output = chain.invoke(query)
print(output)

肌ラボ 極潤ヒアルロン液の使用上の注意点は以下の通りです：

1. **異常に注意**: 使用中または使用後に、赤み、はれ、かゆみ、刺激、色抜け（白斑等）や黒ずみ等の異常が現れた場合は、使用を中止し、皮膚科専門医等に相談すること。

2. **異常のある部位には使用しない**: 傷、はれもの、湿疹等、異常のある部位には使用しないこと。

3. **目に入らないように注意**: 目に入った場合は、すぐに水またはぬるま湯で洗い流し、異常が残る場合は眼科医に相談すること。
